In [3]:
import pandas as pd
import numpy as np
import joblib
import os   # ← THIS WAS MISSING

In [4]:
import pandas as pd
import numpy as np
import joblib

In [5]:
df = pd.read_csv("datasets/perovai_week05_day2_predictions.csv")
anoms = pd.read_csv("datasets/perovai_week05_day4_anomalies.csv")
imps = pd.read_csv("datasets/perovai_week05_day3_feature_importance.csv", header=None)

model = joblib.load("datasets/perovai_best_model_calibrated.pkl") if \
    os.path.exists("datasets/perovai_best_model_calibrated.pkl") else joblib.load("datasets/perovai_best_model_week05.pkl")

df.head()

,Device,FF_Forward,FF_Reverse,Jsc_mAcm2_Forward,Jsc_mAcm2_Reverse,PCE_percent_Forward,PCE_percent_Reverse,Rs_ohm_cm2_Forward,Rs_ohm_cm2_Reverse,Rsh_ohm_cm2_Forward,...,Delta_PCE,Delta_Voc,Delta_Jsc,Delta_FF,Hysteresis_Index,High_Hysteresis,Low_Rs,Unstable,Predicted_Instability_Prob,Predicted_Label
0,device001,0.650051,0.640161,20.177047,20.178596,14.491762,14.377952,8.245944,8.304344,5106.928620,...,-0.113810,0.008172,0.001549,-0.009890,-0.007916,0,1,0,1.451066e-07,0
1,device002,0.746704,0.758766,20.051937,20.154208,17.134683,17.777031,6.118376,7.594258,3341.636255,...,0.642347,0.018098,0.102270,0.012062,0.036134,0,1,0,1.432962e-07,0
2,device003,0.771115,0.767192,20.756446,20.797109,18.363962,18.387340,4.786209,5.584745,4110.186343,...,0.023378,0.005078,0.040663,-0.003923,0.001271,0,1,0,1.451066e-07,0
3,device004,0.748184,0.759476,20.071712,20.171704,17.191299,17.810883,6.110401,7.581881,3240.772843,...,0.619584,0.017833,0.099991,0.011292,0.034787,0,1,0,1.451066e-07,0
4,device005,0.748388,0.755708,20.152430,20.259536,17.333143,17.911775,6.452673,8.205573,3485.020804,...,0.578632,0.020645,0.107106,0.007320,0.032305,0,1,0,1.451066e-07,0


In [6]:
df = df.merge(
    anoms[["Device","Anomaly"]],
    on="Device",
    how="left"
)
df["Anomaly"] = df["Anomaly"].fillna(0).astype(int)

In [7]:
def risk_tier(p):
    if p >= 0.7:
        return "High"
    elif p >= 0.4:
        return "Medium"
    else:
        return "Low"

df["Risk_Tier"] = df["Predicted_Instability_Prob"].apply(risk_tier)
df["Risk_Tier"].value_counts()

Risk_Tier
Low       54
High      50
Medium     2
Name: count, dtype: int64

In [8]:
df = df.sort_values("Predicted_Instability_Prob", ascending=False)
df[["Device","Predicted_Instability_Prob","Risk_Tier","Anomaly"]].head(15)

,Device,Predicted_Instability_Prob,Risk_Tier,Anomaly
7,device008,1.0,High,1
9,device010,1.0,High,0
21,device022,1.0,High,0
25,device026,1.0,High,0
24,device025,1.0,High,1
41,device042,1.0,High,0
22,device023,1.0,High,0
45,device046,1.0,High,0
44,device045,1.0,High,0
42,device043,1.0,High,0


In [9]:
features = [
    "Voc_V_Reverse",
    "Jsc_mAcm2_Reverse",
    "FF_Reverse",
    "Rs_ohm_cm2_Reverse",
    "Rsh_ohm_cm2_Reverse",
    "Delta_PCE",
    "Hysteresis_Index"
]

X = df[features]
contrib = X * model.base_estimator_.coef_[0] if hasattr(model, "base_estimator_") else X

df["Top_Driver"] = contrib.abs().idxmax(axis=1)

In [10]:
final = df[[
    "Device",
    "Predicted_Instability_Prob",
    "Risk_Tier",
    "Top_Driver",
    "Anomaly",
    "Voc_V_Reverse",
    "Jsc_mAcm2_Reverse",
    "FF_Reverse",
    "Rs_ohm_cm2_Reverse",
    "Rsh_ohm_cm2_Reverse",
    "Delta_PCE",
    "Hysteresis_Index"
]]

final.head(10)

,Device,Predicted_Instability_Prob,Risk_Tier,Top_Driver,Anomaly,Voc_V_Reverse,Jsc_mAcm2_Reverse,FF_Reverse,Rs_ohm_cm2_Reverse,Rsh_ohm_cm2_Reverse,Delta_PCE,Hysteresis_Index
7,device008,1.0,High,Rs_ohm_cm2_Reverse,1,1.219800,-0.000019,-0.606497,2.026717e+08,2.381029e+06,4.859762e-06,0.337591
9,device010,1.0,High,Rsh_ohm_cm2_Reverse,0,0.684069,0.008918,0.541720,1.571928e+04,3.474437e+05,4.986416e-04,0.150891
21,device022,1.0,High,Rsh_ohm_cm2_Reverse,0,0.781325,17.895528,0.424888,1.511822e+01,1.091839e+02,1.028520e+00,0.173126
25,device026,1.0,High,Rsh_ohm_cm2_Reverse,0,0.810258,17.882367,0.436219,1.461828e+01,1.202947e+02,1.130071e+00,0.178794
24,device025,1.0,High,Rs_ohm_cm2_Reverse,1,0.476894,0.000006,0.330804,7.367866e+07,2.880276e+07,6.498116e-07,0.671607
41,device042,1.0,High,Rsh_ohm_cm2_Reverse,0,1.079738,19.062989,0.558070,1.045415e+01,3.382840e+02,2.424966e+00,0.211110
22,device023,1.0,High,Rsh_ohm_cm2_Reverse,0,0.581305,16.517321,0.381998,1.564707e+01,7.622869e+01,4.192897e-01,0.114317
45,device046,1.0,High,Rsh_ohm_cm2_Reverse,0,1.067190,19.028035,0.552177,9.313728e+00,3.147212e+02,1.125685e+00,0.100393
44,device045,1.0,High,Rsh_ohm_cm2_Reverse,0,1.032476,19.241814,0.468945,1.231424e+01,1.578626e+02,1.264251e+00,0.135702
42,device043,1.0,High,Jsc_mAcm2_Reverse,0,0.356715,18.478478,0.264963,1.702347e+01,1.402707e+01,7.230662e-01,0.414004


In [11]:
final.to_csv("datasets/perovai_week05_day5_decision_table.csv", index=False)
print("Saved: datasets/perovai_week05_day5_decision_table.csv")

Saved: datasets/perovai_week05_day5_decision_table.csv
